**Importing the neccessary libraries**

In [2]:
import pandas as pd
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize
print(os.listdir("../input/chest-xray-pneumonia/chest_xray/chest_xray/"))

['train', '.DS_Store', 'val', 'test']


In [3]:
print(os.listdir("../input/chest-xray-pneumonia/chest_xray/chest_xray/train/"))

['PNEUMONIA', 'NORMAL', '.DS_Store']


**Saving the paths for later use**

In [4]:
TRAIN_DIR = "../input/chest-xray-pneumonia/chest_xray/chest_xray/train/"
TEST_DIR =  "../input/chest-xray-pneumonia/chest_xray/chest_xray/test/"

In [5]:
def get_label(Dir):
    for nextdir in os.listdir(Dir):
        if not nextdir.startswith('.'):
            if nextdir in ['NORMAL']:
                label = 0
            elif nextdir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
    return nextdir, label

In [6]:
def preprocessing_data(Dir):
    X = []
    y = []
    
    for nextdir in os.listdir(Dir):
        nextdir, label = get_label(Dir)
        temp = Dir + nextdir
        
        for image_filename in tqdm(os.listdir(temp)):
            path = os.path.join(temp + '/' , image_filename)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = skimage.transform.resize(img, (150, 150, 3))
                img = np.asarray(img)
                X.append(img)
                y.append(label)
            
    X = np.asarray(X)
    y = np.asarray(y)
    
    return X,y

In [40]:
def fuck():
    print(x)

In [7]:
def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['NORMAL']:
                label = 0
            elif nextDir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                img = cv2.imread(temp + '/' + file)
                if img is not None:
                    img = skimage.transform.resize(img, (150, 150, 3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(299, 299, 3))
                    img = np.asarray(img)
                    X.append(img)
                    y.append(label)
                    
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

**Loading the dataset**

In [8]:
X_train, y_train = get_data(TRAIN_DIR)

100%|██████████| 1342/1342 [10:20<00:00,  2.16it/s]


In [9]:
X_test , y_test = get_data(TEST_DIR)

100%|██████████| 234/234 [01:59<00:00,  1.95it/s]


In [10]:
print(X_train.shape,'\n',X_test.shape)

(5216, 150, 150, 3) 
 (624, 150, 150, 3)


In [34]:
print(X_train.shape,'\n',X_test.shape)

(5216, 150, 150, 3) 
 (624, 150, 150, 3)


In [35]:
print(y_train.shape,'\n',y_test.shape)

(5216, 2) 
 (624, 2)


**There are two possible classes of output**

In [12]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

Using TensorFlow backend.


In [13]:
Pimages = os.listdir(TRAIN_DIR + "PNEUMONIA")
Nimages = os.listdir(TRAIN_DIR + "NORMAL")

In [24]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
#K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

**Loading the Inception model**

In [31]:
from keras.applications.inception_v3 import InceptionV3
# create the base pre-trained model
base_model = InceptionV3(weights=None, include_top=False , input_shape=(150, 150,3))

In [36]:
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(2, activation='sigmoid')(x)

In [37]:
model = Model(inputs=base_model.input, outputs=predictions)

**Compiling the model**

In [38]:
model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

**Finally training the model**

In [39]:
batch_size = 64
epochs = 10
history = model.fit(X_train, y_train, validation_data = (X_test , y_test)  ,
          epochs=epochs)

Train on 5216 samples, validate on 624 samples
Epoch 1/10
5216/5216 [==============================] - 122s 23ms/step - loss: 0.2374 - accuracy: 0.8909 - val_loss: 3.7393 - val_accuracy: 0.6250
Epoch 2/10
5216/5216 [==============================] - 46s 9ms/step - loss: 0.1433 - accuracy: 0.9477 - val_loss: 3.1569 - val_accuracy: 0.6250
Epoch 3/10
5216/5216 [==============================] - 46s 9ms/step - loss: 0.1089 - accuracy: 0.9565 - val_loss: 2.0492 - val_accuracy: 0.6250
Epoch 4/10
5216/5216 [==============================] - 46s 9ms/step - loss: 0.0905 - accuracy: 0.9684 - val_loss: 1.0348 - val_accuracy: 0.7468
Epoch 5/10
5216/5216 [==============================] - 46s 9ms/step - loss: 0.0783 - accuracy: 0.9718 - val_loss: 0.8997 - val_accuracy: 0.7724
Epoch 6/10
5216/5216 [==============================] - 46s 9ms/step - loss: 0.0765 - accuracy: 0.9739 - val_loss: 1.5024 - val_accuracy: 0.7228
Epoch 7/10
5216/5216 [==============================] - 48s 9ms/step - loss: 0.07

In [51]:
from sklearn.metrics import confusion_matrix
pred = model.predict(X_test)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

In [54]:
CM = confusion_matrix(y_true, pred)

In [ ]:
CM